## Setup

In [1]:
import os
import requests
import json
from pathlib import Path

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [2]:
load_eoepca_state()

In [3]:
platform_domain = os.environ.get("INGRESS_HOST")
resource_discovery_domain = f'{os.environ.get("HTTP_SCHEME")}://resource-catalogue.{platform_domain}'

print(f"Resource Discovery URL: {resource_discovery_domain}")

Resource Discovery URL: https://resource-catalogue.notebook-test.develop.eoepca.org


## Validate Resource Discovery Endpoints

In [4]:
endpoints = [
    ("Landing Page", resource_discovery_domain),
    ("Swagger UI", f"{resource_discovery_domain}/openapi?f=html"),
    ("Collections", f"{resource_discovery_domain}/collections"),
    ("Conformance", f"{resource_discovery_domain}/conformance"),
    ("CSW GetCapabilities", f"{resource_discovery_domain}/csw?service=CSW&version=2.0.2&request=GetCapabilities"),
    ("STAC", f"{resource_discovery_domain}/stac")
]

for name, url in endpoints:
    response = requests.get(url)
    print(f"{name} ({url}): {response.status_code}")

Landing Page (https://resource-catalogue.notebook-test.develop.eoepca.org): 200
Swagger UI (https://resource-catalogue.notebook-test.develop.eoepca.org/openapi?f=html): 200
Collections (https://resource-catalogue.notebook-test.develop.eoepca.org/collections): 200
Conformance (https://resource-catalogue.notebook-test.develop.eoepca.org/conformance): 200
CSW GetCapabilities (https://resource-catalogue.notebook-test.develop.eoepca.org/csw?service=CSW&version=2.0.2&request=GetCapabilities): 200
STAC (https://resource-catalogue.notebook-test.develop.eoepca.org/stac): 200


## STAC Items Search

In [5]:
search_url = f"{resource_discovery_domain}/stac/search"
search_payload = {
    "bbox": [-10, 35, 40, 70],
    "datetime": "2020-01-01T00:00:00Z/2025-12-31T23:59:59Z",
    "limit": 3
}

response = requests.post(search_url, json=search_payload)
print(f"STAC search status code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

STAC search status code: 200
{
  "type": "FeatureCollection",
  "features": [],
  "links": [
    {
      "rel": "self",
      "type": "application/geo+json",
      "title": "This document as GeoJSON",
      "href": "https://resource-catalogue.notebook-test.develop.eoepca.org/stac/collections/metadata:main/items?limit=3&bbox=-10%2C35%2C40%2C70&datetime=2020-01-01T00%3A00%3A00Z%2F2025-12-31T23%3A59%3A59Z&f=json",
      "hreflang": "en-US"
    },
    {
      "rel": "collection",
      "type": "application/json",
      "title": "Collection URL",
      "href": "https://resource-catalogue.notebook-test.develop.eoepca.org/stac/collections/metadata:main",
      "hreflang": "en-US"
    },
    {
      "rel": "root",
      "type": "application/json",
      "href": "https://resource-catalogue.notebook-test.develop.eoepca.org/stac/"
    }
  ],
  "numberMatched": 0,
  "numberReturned": 0,
  "distributedFeatures": []
}


## Ingesting and Validating a Sample Record

### Load and Inspect Sample Record

In [9]:
sample_record_path = Path("sample_record.xml") # You will see this very simple sample file locally, feel free to experiment with it.

if not sample_record_path.exists():
    sample_record_url = "https://raw.githubusercontent.com/EOEPCA/deployment-guide/notebooks/examples/resource-discovery/sample_record.xml"
    response = requests.get(sample_record_url)
    sample_record_path.write_text(response.text)

print(sample_record_path.read_text()[:500])

<?xml version="1.0" encoding="UTF-8"?>
<csw:Transaction service="CSW" version="2.0.2"
    xmlns:csw="http://www.opengis.net/cat/csw/2.0.2"
    xmlns:ows="http://www.opengis.net/ows"
    xmlns:dc="http://purl.org/dc/elements/1.1/"
    xmlns:dct="http://purl.org/dc/terms/">
  <csw:Insert>
    <csw:Record>
      <dc:identifier>urn:uuid:12345</dc:identifier>
      <dc:type>http://purl.org/dc/dcmitype/Image</dc:type>
      <dc:format>image/svg+xml</dc:format>
      <dc:title>EOEPCA Sample Record From


### Ingest Sample Records

In [10]:
ingest_url = f"{resource_discovery_domain}/csw"
ingest_headers = {'Content-Type': 'application/xml'}

with open(sample_record_path, 'r') as file:
    record_xml = file.read()

response = requests.post(
    ingest_url,
    params={"request": "Transaction", "service": "CSW", "version": "2.0.2"},
    data=record_xml,
    headers=ingest_headers
)

print(f"Ingestion status code: {response.status_code}")
print(response.text[:500])  # Response snippet

Ingestion status code: 200
<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!-- pycsw 3.0-dev -->
<csw:TransactionResponse xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dct="http://purl.org/dc/terms/" xmlns:gmd="http://www.isotc211.org/2005/gmd" xmlns:gml="http://www.opengis.net/gml" xmlns:gml32="http://www.opengis.net/gml/3.2" xmlns:ows="http://www.opengis.net/ows" xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" 


### Verify Record Ingestion

In [12]:
metadata_collection_url = f"{resource_discovery_domain}/collections/metadata:main/items"
response = requests.get(metadata_collection_url)

items = response.json().get("features", [])
print(items)
ingested_titles = [item["properties"].get("title", "") for item in items]

if "EOEPCA Sample Record" in ingested_titles:
    print("✅ Sample record successfully ingested and discoverable!")
    print('View the record in the catalogue:', f"{resource_discovery_domain}/collections/metadata:main/items")
else:
    print("❌ Sample record ingestion failed or record not discoverable.")

[{'id': 'urn:uuid:19887a8a-f6b0-4a63-ae56-7fba0e17801j', 'type': 'Feature', 'geometry': None, 'properties': {'updated': '2025-03-21T15:21:35Z', 'type': 'http://purl.org/dc/dcmitype/Image', 'title': 'EOEPCA Sample Record', 'description': 'This is a sample abstract.', 'formats': [{'name': 'image/svg+xml'}], 'keywords': ['EOEPCA-TEST']}, 'links': [{'rel': 'self', 'type': 'application/geo+json', 'title': 'urn:uuid:19887a8a-f6b0-4a63-ae56-7fba0e17801j', 'name': 'item', 'description': 'urn:uuid:19887a8a-f6b0-4a63-ae56-7fba0e17801j', 'href': 'https://resource-catalogue.notebook-test.develop.eoepca.org/collections/metadata:main/items/urn:uuid:19887a8a-f6b0-4a63-ae56-7fba0e17801j'}, {'rel': 'collection', 'type': 'application/json', 'title': 'Collection', 'name': 'collection', 'description': 'Collection', 'href': 'https://resource-catalogue.notebook-test.develop.eoepca.org/collections/metadata:main'}], 'time': {'null': 'date'}}, {'id': 'urn:uuid:19887a8a-f6b0-4a63-ae56-7fba0e12801j', 'type': 'Fe